In [29]:
import json
import requests
import re,string,timeit
import time
import pandas as pd
import os
from dotenv import load_dotenv
import numpy as np

Setup your parameters
Request URL
https://api.domain.com.au/v2/suburbPerformanceStatistics/NSW/Castle%20Hill/2154?propertyCategory=house&bedrooms=3&periodSize=quarters&startingPeriodRelativeToCurrent=40&totalPeriods=40


Provide your client credentials as per your Domain developer account.

Required: client_id and client_secret

Make a POST request to receive token.

In [33]:
client_id = 'client_405e33d0d19197c377851f252dbfa5ff' 
client_secret = 'secret_461901bea416e4731f15dc40467ff8e8'

In [34]:
# load_dotenv()
# client_id =os.getenv("domain_client_id")
# client_secret = os.getenv("domain_client_oauth_secret")



In [35]:
print(client_secret)

secret_461901bea416e4731f15dc40467ff8e8


In [36]:
#to authenticate client credentials
# Make a POST request to the https://auth.domain.com.au/v2/connect/token endpoint using your client_id and client_secret, along with the list of scopes that you require. 
auth_url2 = 'https://auth.domain.com.au/v1/connect/token'

In [37]:

# https://developer.domain.com.au/docs/v2/apis/pkg_properties_locations/references/suburbperformance_get_bynamedsuburb
# GET /v2/suburbPerformanceStatistics/{state}/{suburb}/{postcode}

In [38]:
#Scope and Permissions
#The following scopes may be used when calling this endpoint
#https://developer.domain.com.au/docs/v1/apis/pkg_properties_locations/references/properties_get:

scopes = ['api_suburbperformance_read']

In [40]:
url_endpoint2 = 'https://api.domain.com.au/v2/suburbPerformanceStatistics/'

#to define suburb details example for 1 suburb only
state='NSW'
suburb="Chatswood"
postcode='2067'
# property_category = 'house'
period_size = 'quarters'
period_length = 40
total_periods= 40
url_detailed=f'https://api.domain.com.au/v2/suburbPerformanceStatistics/{state}/{suburb}/{postcode}/'


# url_detailed = 'https://api.domain.com.au/v2/suburbPerformanceStatistics/{}/{}/{}?propertyCategory={}&bedrooms={}&periodSize={}&startingPeriodRelativeToCurrent={}&totalPeriods={}'.format(state,suburb,postcode,propertyCategory,bedrooms,periodSize,startingPeriodRelativeToCurrent,totalPeriods)

In [41]:

df=pd.read_csv("./Resources/nsw_data.csv",index_col="Suburb")


In [42]:
df


,Postcode
Suburb,
Sydney,1001
Sydney,1002
Sydney,1003
Sydney,1004
Sydney,1005
Sydney,1006
Sydney,1007
Sydney,1008
Sydney,1009


In [43]:
mask1=df['Postcode']>1999
list_suburbs=df[mask1]
list_suburbs=list_suburbs.loc[np.random.choice(list_suburbs.index,100,replace=False)]
list_suburbs=list_suburbs.to_dict()["Postcode"]
list_suburbs
# len(list_suburbs)

{'Huntingwood': 2148,
 'Bungonia': 2580,
 'Horsley Park': 2175,
 'Murrami': 2705,
 'Frederickton': 2440,
 'Primrose Valley': 2621,
 'Merewether': 2291,
 'Brenda': 2831,
 'Moonee Beach': 2450,
 'Alexandria': 2015,
 'Kundle Kundle': 2430,
 'South Gundurimba': 2480,
 'Georges Creek': 2440,
 'Clover Park': 2469,
 'Doyalson': 2262,
 'Fairfield West': 2165,
 'Chilcotts Grass': 2480,
 'Bland': 2721,
 'Sapphire': 2360,
 'Dorrigo North': 2453,
 'Wallaga Lake Heights': 2546,
 'Pooncarie': 2648,
 'Blair Athol': 2560,
 'The Scrub': 2372,
 'Bathurst': 2795,
 'Bay Village': 2261,
 'Allambie Heights': 2100,
 'Steinbrook': 2372,
 'Commissioners Waters': 2350,
 'Urangeline East': 2656,
 'Bungulla': 2372,
 'Mount Wilson': 2786,
 'Kingscliff': 2487,
 'Murrumburrah': 2587,
 'Nulkaba': 2325,
 'Yagoona': 2199,
 'Marsfield': 2122,
 'Swanbrook': 2360,
 'Edgeworth': 2285,
 'Teven': 2478,
 'Lowesdale': 2646,
 'Grassy Head': 2441,
 'Pyangle': 2849,
 'Jones Creek': 2722,
 'Box Hill': 2765,
 'Thredbo': 2625,
 'Wea

In [44]:
final_list_suburbs = []

In [45]:
#housing data with bedrooms, houseCategories, timeperiods length
def get_property_info(suburb,postcode):
    response = requests.post(auth_url2,
        data = {
        'client_id':client_id,
        'client_secret':client_secret,
        'grant_type':'client_credentials',
        'scope': scopes,
        'Content-Type':'text/json'}
        )
    json_res = response.json()
    access_token=json_res["access_token"]
    print(access_token)
    auth = {'Authorization':'Bearer '+access_token}
    
    url_detailed=f'https://api.domain.com.au/v2/suburbPerformanceStatistics/NSW/{suburb}/{postcode}/?totalPeriods=40'



    res2 = requests.get(url_detailed,headers = auth)
    r2=res2.json()
    json_object =json.dumps(r2,indent=4)
    json.dumps(r2,indent=4)
        
        

    
    if "header"  in json.loads(json_object).keys(): 

        with open("./resources/{}_{}.json".format(postcode,suburb),"w") as outfile:
            outfile.write(json_object)
        
        
    else:

        print('Failure in',suburb, postcode,e,access_token)






In [46]:
#  final_list_suburbs.append(suburb)
for suburb,postcode in list_suburbs.items():
    try:
        get_property_info(suburb,postcode)
        print('Success', suburb, postcode)
        final_list_suburbs.append(suburb)

    except Exception as e:
       
        print('Failure in',suburb, postcode,e)
        pass
    
   

0f32eec1e300743299043f71331fe9eb
Success Huntingwood 2148
66ed90df7ecd87e695f9e049177131ad
Success Bungonia 2580
56d5d944ac7f071bfc798bb9638f452b
Success Horsley Park 2175
140c590f33fb93b2ccc28d4df6569976
Success Murrami 2705
0605226e8b469c7b02148a01897c725d
Success Frederickton 2440
d5f1990b8b877dcd7d3515683bc48c4a
Success Primrose Valley 2621
e85a9460d005ab514d37da5e4790b6b0
Success Merewether 2291
a7a4964bd41aa9a1f3779abd726cff9b
Failure in Brenda 2831 Expecting value: line 1 column 1 (char 0)
30b573feb5c4287508df2cf503fa83f0
Success Moonee Beach 2450
a73c1262544c506e5775ff252051d48d
Success Alexandria 2015
5fa7a5f3312dfc82db4206ed56c65c1b
Success Kundle Kundle 2430
cfdf5f3b0170632b6c78abe612f40f86
Success South Gundurimba 2480
80273fdee3c3618ecef498dd913a3db9
Failure in Georges Creek 2440 Expecting value: line 1 column 1 (char 0)
a22c6983c6885008a49194a8d4cb9f2a
Failure in Clover Park 2469 Expecting value: line 1 column 1 (char 0)
a0887c40094281e821089b2969b58ef7
Success Doyalson 2

In [47]:
len(final_list_suburbs)

75

In [48]:
#to open json file saved in previous step & convert to dataframe & combine all dataframe into a master dataframe
df = pd.DataFrame()
frames= []
for suburb in final_list_suburbs: 
    with open("./resources/{}_{}.json".format(list_suburbs[suburb],suburb),"r") as sub_housing_data:
        sub_housing_data=json.load(sub_housing_data)
        suburb_name=sub_housing_data['header']['suburb']
        
    for x in range (40):
        
        headers_dict1=sub_housing_data['series']['seriesInfo'][x]['values']
        headers_dict1["year"]=sub_housing_data['series']['seriesInfo'][x]["year"]
        headers_dict1["month"]=sub_housing_data['series']['seriesInfo'][x]["month"]
        sub_df1 = pd.DataFrame(headers_dict1.values(),index=headers_dict1.keys(),columns=[suburb_name]).T
      
        sub_df1 = sub_df1.reset_index().drop(columns=["auctionNumberWithdrawn","daysOnMarket","discountPercentage",
                                                      "5thPercentileSoldPrice","25thPercentileSoldPrice",
                                                      "75thPercentileSoldPrice","95thPercentileSoldPrice",
                                                      "numberRentListing","highestRentListingPrice","lowestRentListingPrice"]).rename(columns={"index":"suburb"})
        frames.append(sub_df1)
df = pd.concat(frames)
        

    

In [49]:
df.head(10)

,suburb,medianSoldPrice,numberSold,highestSoldPrice,lowestSoldPrice,medianSaleListingPrice,numberSaleListing,highestSaleListingPrice,lowestSaleListingPrice,auctionNumberAuctioned,auctionNumberSold,medianRentListingPrice,year,month
0,Huntingwood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011.0,9.0
0,Huntingwood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011.0,12.0
0,Huntingwood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012.0,3.0
0,Huntingwood,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2012.0,6.0
0,Huntingwood,NaN,NaN,NaN,NaN,NaN,1.0,5500000.0,5500000.0,1.0,NaN,NaN,2012.0,9.0
0,Huntingwood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2012.0,12.0
0,Huntingwood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013.0,3.0
0,Huntingwood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013.0,6.0
0,Huntingwood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013.0,9.0
0,Huntingwood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2013.0,12.0


In [50]:
df.columns

Index(['suburb', 'medianSoldPrice', 'numberSold', 'highestSoldPrice',
       'lowestSoldPrice', 'medianSaleListingPrice', 'numberSaleListing',
       'highestSaleListingPrice', 'lowestSaleListingPrice',
       'auctionNumberAuctioned', 'auctionNumberSold', 'medianRentListingPrice',
       'year', 'month'],
      dtype='object')

In [51]:
df = df[['year', 'month','suburb', 'medianSoldPrice', 'numberSold', 'highestSoldPrice',
       'lowestSoldPrice', 'medianSaleListingPrice', 'numberSaleListing',
       'highestSaleListingPrice', 'lowestSaleListingPrice',
       'auctionNumberAuctioned', 'auctionNumberSold', 'medianRentListingPrice',
       ]]

In [52]:
df.sample(10)

,year,month,suburb,medianSoldPrice,numberSold,highestSoldPrice,lowestSoldPrice,medianSaleListingPrice,numberSaleListing,highestSaleListingPrice,lowestSaleListingPrice,auctionNumberAuctioned,auctionNumberSold,medianRentListingPrice
0,2019.0,6.0,Kingscliff,923000.0,36.0,2625000.0,265000.0,840000.0,94.0,3395000.0,10000.0,7.0,2.0,723.0
0,2012.0,12.0,Yagoona,518000.0,53.0,728000.0,364000.0,504000.0,87.0,1380000.0,159000.0,31.0,21.0,430.0
0,2018.0,9.0,South Gundurimba,NaN,1.0,560000.0,560000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2020.0,9.0,Quakers Hill,798000.0,107.0,1227000.0,481000.0,755000.0,181.0,1400000.0,413000.0,3.0,2.0,460.0
0,2020.0,3.0,Budden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2021.0,3.0,Doyalson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2020.0,6.0,Martins Creek,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2017.0,9.0,Bowman Farm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2016.0,12.0,Frederickton,NaN,1.0,450000.0,450000.0,NaN,4.0,450000.0,349000.0,NaN,NaN,NaN
0,2020.0,3.0,Box Hill,511000.0,78.0,3040000.0,415000.0,776000.0,656.0,45000000.0,15000.0,NaN,NaN,570.0


In [53]:
df.to_csv('data_25082021_3.csv',index=False)